**cvambio de los datos climaticos**

In [4]:
# =========================
# 1. Importaciones
# =========================
import pandas as pd
import numpy as np
import re
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import plotly.graph_objs as go
import plotly.express as px
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

# =========================
# 2. Carga de datos de celdas
# =========================
carpeta_xls = "C:\\Users\\Freddy\\OneDrive - CELSIA S.A E.S.P\\Trabajo Actual\\Personal\\UNAD\\2025\\Especializacion\\Cursos\\Poyecto de grado II\\Recopilacion"
archivos = [f for f in os.listdir(carpeta_xls) if f.endswith(".xls")]

registros_verticales = []
for archivo in archivos:
    ruta = os.path.join(carpeta_xls, archivo)
    nombre_archivo = os.path.splitext(archivo)[0]
    fecha_match = re.search(r"(\d{2})_(\d{4})", nombre_archivo)
    if fecha_match:
        mes, año = fecha_match.groups()
        fecha_columna = f"{mes}/{año}"
    else:
        fecha_columna = nombre_archivo

    try:
        df = pd.read_excel(ruta, sheet_name=0, engine='xlrd')
        df.columns = [str(col).strip() for col in df.columns]
        for i in range(0, len(df.columns), 3):
            grupo = df.iloc[:, i:i+3].copy()
            grupo.columns = ["Celda N°", "Tensión (V)", "Gravedad Específica"]
            grupo = grupo[grupo["Celda N°"].apply(lambda x: str(x).strip().isdigit())]
            grupo["Celda N°"] = pd.to_numeric(grupo["Celda N°"], errors="coerce")
            grupo["Tensión (V)"] = grupo["Tensión (V)"].astype(str).str.replace(",", ".").astype(float)
            grupo["Gravedad Específica"] = pd.to_numeric(grupo["Gravedad Específica"], errors="coerce")
            grupo["Fecha"] = fecha_columna
            grupo = grupo[grupo["Celda N°"].between(1, 60)]
            registros_verticales.append(grupo)
    except Exception as e:
        print(f" Error procesando {archivo}: {e}")

df_vertical = pd.concat(registros_verticales, ignore_index=True)
df_vertical["Fecha_dt"] = pd.to_datetime("01/" + df_vertical["Fecha"], format="%d/%m/%Y", errors="coerce")
df_vertical = df_vertical.sort_values(["Fecha_dt", "Celda N°"]).reset_index(drop=True)

# =========================
# 3. Variables climáticas (desde CSV)
# =========================
df = pd.read_csv("C:\\Users\\Freddy\\OneDrive - CELSIA S.A E.S.P\\Trabajo Actual\\Personal\\UNAD\\2025\\Especializacion\\Cursos\\Poyecto de grado II\\Recopilacion\\descargaDhime.csv")

df = df[['Variable', 'Valor', 'Fecha']]
df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')
df['mes'] = df['Fecha'].dt.to_period('M')

promedios = df.groupby(['Variable', 'mes'])['Valor'].mean().reset_index()
prom_temp = promedios[promedios['Variable'].str.lower().str.contains("temp")]


# =========================
# 5. Dashboard con Dash
# =========================
app = Dash(__name__)
app.layout = html.Div([
    html.H1("Dashboard de Evolución Celdas, Gravedad Específica, Temperatura y Tensión - Amaime, Valle del Cauca"),

    # Evolución por celda
    dcc.Dropdown(
        id="celda_selector",
        options=[{"label": c, "value": c} for c in df_vertical["Celda N°"].unique()],
        value=df_vertical["Celda N°"].unique()[0]
    ),
    dcc.Graph(id="evolucion_celda"),

   
        
    
])

# =========================
# Callback con ejes separados (Gravedad lineal, Temperatura log, Tensión secundario)
# =========================
@app.callback(
    Output("evolucion_celda", "figure"),
    [Input("celda_selector", "value")]
)
def update_graph(celda_id):
    df_cell = df_vertical[df_vertical["Celda N°"] == celda_id]

    fig = go.Figure()

    # Gravedad Específica (eje primario lineal)
    fig.add_trace(go.Scatter(
        x=df_cell["Fecha_dt"],
        y=df_cell["Gravedad Específica"],
        mode="lines+markers",
        name="Gravedad Específica",
        yaxis="y1",
        line=dict(color="green")
    ))

    # Temperatura promedio mensual (eje logarítmico separado)
    fig.add_trace(go.Scatter(
        x=prom_temp["mes"].astype(str),
        y=prom_temp["Valor"],
        mode="lines+markers",
        name="Temperatura (°C)",
        yaxis="y3",
        line=dict(color="red")
    ))

    # Tensión (eje secundario lineal)
    fig.add_trace(go.Scatter(
        x=df_cell["Fecha_dt"],
        y=df_cell["Tensión (V)"],
        mode="lines+markers",
        name="Tensión (V)",
        yaxis="y2",
        line=dict(color="blue")
    ))

    fig.update_layout(
        title=f"Evolución Celda {celda_id}: Gravedad Específica, Temperatura (log) y Tensión",
        xaxis=dict(title="Mes/Año"),
        yaxis=dict(
            title="Gravedad Específica",
            side="left",
            type="linear",
            position=0.0
        ),
        yaxis2=dict(
            title="Tensión (V)",
            overlaying="y",
            side="right",
            type="linear",
            position=1.0
        ),
        yaxis3=dict(
            title="Temperatura (°C)",
            overlaying="y",
            side="left",
            type="log",
            position=0.07   # desplazado para evitar superposición
        ),
        legend=dict(x=0.01, y=0.99)
    )

    return fig

if __name__ == "__main__":
    app.run(debug=True)

